# Simulating a FHN network.

## Set parameters and define the network

In [207]:
include("src/network_matrices_creation.jl")
include("src/network_simulation.jl")
using Random
N = 90
eps = 0.05
a = 0.5
b = -bmatrix(pi/2-0.1, eps)
σ = 0.0506
G = wattsstrogatzmatrix(N, 3, 1);#0.232)


In [208]:
# Set the initial conditions

x_0 = zeros(2*N)
x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3) #rand(Uniform(-a + a^3 / 3, a - a^3 / 3), N)

# Define the ODE problem
prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 1000.0), [a, eps, σ])

# Solve the ODE problem
# alg = Tsit5()
sol = solve(prob; dtmax=0.1);


InterruptException: InterruptException:

### Plot the kuramoto order parameter

In [210]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (1500, 1000))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f


┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie /Users/miguel/.julia/packages/Makie/fyNiH/src/scenes.jl:220


BoundsError: BoundsError: attempt to access 20×800003 Matrix{Float64} at index [21, 1:800003]

## Study Cluster Synchronization in the integration of the FHN model.

### First, a test

Based on what is shown in the paper (https://doi.org/10.48550/arXiv.2303.08668), first we test with a complete graph.

In [211]:
include("src/msf.jl")
zero_msf = msf_zero()
println("zero_msf = ", zero_msf)
println("msf = ", master_stability_function(zero_msf, 0))

zero_msf = 0.1954037965279418
msf = -4.10727451510246e-17


In [212]:
include("src/network_matrices_creation.jl")
include("src/network_simulation.jl")
include("src/cluster_synch.jl")
using Random
N = 10
eps = 0.05
a = 0.5
b = -bmatrix(pi/2-0.1, eps)
G = test_matrix_for_cluster_synch();
eigenvalues, eigenvectors, clusters, s_matrices = s_matrix_method(G)
critical_couplings = unique(round.(zero_msf./eigenvalues, digits=8))[2:end]
println("Critical Couplings: ", critical_couplings)
σ = critical_couplings[end-4]*0.9

Critical Couplings: [0.019540379652794187, 0.019540379652794152, 0.011494340972231877, 0.011494340972231875, 0.011494340972231865, 0.009304942691806761, 0.009304942691806754, 0.009304942691806753]


0.010344906875008688

In [213]:
# Set the initial conditions

x_0 = zeros(2*N)
# x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
# x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3)

# Define the ODE problem
prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 1000.0), [a, eps, σ])

# Solve the ODE problem
# alg = Tsit5()
sol = solve(prob; dtmax=0.5);


In [217]:
using GLMakie
include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

f = Figure(size= (800, 600))

i = 0
for cluster in unique(clusters)
    if isempty(cluster)
        continue
    else
        i += 1
        ax = Axis(f[i, 1])
        ax.xlabel = "Time"
        ax.ylabel = "Synch Error"
        println("Cluster $i: $cluster")
        n_cluster = i
        t_values, synch_error = local_synch_error(sol, cluster)
        lines!(ax, t_values, synch_error; label="Cluster $n_cluster", linewidth=1)
        axislegend()
    end
end
print("There are $(i-1) non-empty clusters")
display(f);

Cluster 1: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Cluster 2: [4, 5, 6, 7, 8, 9, 10]


Cluster 3: [7, 8, 9, 10]
There are 2 non-empty clusters

In [ ]:
using GLMakie
include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

d_sweep = range(0, 0.3, length=20)
synch_averages = zeros(20)

f = Figure(size= (800, 600))

for σ in d_sweep
    x_0 = zeros(2*N) + 0.1 .* rand(2*N) .- 0.05
    # x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
    # x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3)
    
    prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 500.0), [a, eps, σ])
    # alg = Tsit5()
    sol = solve(prob; dtmax=0.5);
    i = 0
    for cluster in unique(clusters)
        if isempty(cluster)
            continue
        else
            i += 1
            println("Cluster $i: $cluster")
        end
        t_values, synch_error = local_synch_error(sol, cluster)
        synch_averages[i] += 
        
    end
end
print("There are $(i-1) non-empty clusters")
display(f);


In [130]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (800, 200))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f

### Now for a watts strogatz network:

In [179]:
include("src/network_matrices_creation.jl")
include("src/network_simulation.jl")
using Random
N = 40
eps = 0.05
a = 0.5
b = -bmatrix(pi/2-0.1, eps)
# σ = 0.012#0.0506
G = wattsstrogatzmatrix(N, 19, 1);#0.232)
eigenvalues, eigenvectors, clusters, s_matrices = s_matrix_method(G)
println("Clusters: ", unique(clusters))
critical_couplings = unique(zero_msf./eigenvalues)[2:end]
println("Critical Couplings: ", critical_couplings)


Clusters: Any[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], [1, 2, 3, 4, 5, 6, 7, 8, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 33, 34, 35, 36, 37, 38, 39, 40], [1, 2, 3, 4, 5, 6, 7, 8, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 25, 26, 27, 29, 33, 34, 35, 36, 37, 39, 40], [1, 2, 3, 4, 5, 6, 8, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 25, 26, 27, 29, 33, 34, 35, 36, 37, 39, 40], [1, 3, 5, 6, 8, 12, 13, 20, 21, 22, 26, 27, 29, 33, 40], [1, 3, 5, 6, 8, 12, 13, 20, 22, 26, 27, 29, 33], [1, 6, 8, 12, 13, 20, 22, 26, 27, 29, 33], [1, 6, 8, 12, 13, 20, 22, 27, 33], Int64[]]
Critical Couplings: [0.005478744, 0.00545365, 0.005295436, 0.005281184, 0.005184652, 0.005179581, 0.005142205, 0.005062993, 0.005010354, 0.004962789, 0.004949288, 0.004917096, 0.004885095]


In [177]:
σ = critical_couplings[end-4]*1.11

# Set the initial conditions

x_0 = zeros(2*N)
# x_0[2 .* (1:N) .- 1] = rand(N) .* 2 .* a .- a
# x_0[2 .* (1:N)] = rand(N) .* 2 .* (-a + a^3 / 3) .- (-a + a^3 / 3)

# Define the ODE problem
prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 1000.0), [a, eps, σ])

# Solve the ODE problem
# alg = Tsit5()
sol = solve(prob; dtmax=0.1);

In [178]:
using GLMakie
include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")

f = Figure(size= (800, 600))

i = 0
for cluster in unique(clusters)
    if isempty(cluster)
        continue
    else
        i += 1
        ax = Axis(f[i, 1])
        ax.xlabel = "Time"
        ax.ylabel = "Synch Error"
        println("Cluster $i: $cluster")
        n_cluster = i
        t_values, synch_error = local_synch_error(sol, cluster)
        lines!(ax, t_values, synch_error; label="Cluster $n_cluster", linewidth=1)
        axislegend()
    end
end
print("There are $(i-1) non-empty clusters")
display(f);

Cluster 1: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]


Cluster 2: [2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
Cluster 3: [3, 6, 7, 9, 12, 13, 14, 15, 17, 18, 19, 20, 21, 25, 26, 28, 30, 33, 34, 36, 37, 38, 39, 40]
Cluster 4: [3, 6, 7, 9, 12, 13, 14, 15, 18, 19, 20, 21, 25, 26, 28, 30, 33, 34, 36, 37, 38, 39, 40]
Cluster 5: [3, 6, 7, 14, 15, 18, 19, 20, 25, 26, 28, 36, 38, 39, 40]


Cluster 6: [3, 6, 7, 14, 15, 18, 19, 26, 28, 39, 40]
There are 5 non-empty clusters

In [175]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(size = (800, 300))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f